In [ ]:

K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

def generate_hash(message: bytearray) -> bytearray:
   

    if isinstance(message, str):
        message = bytearray(message, 'ascii')
    elif isinstance(message, bytes):
        message = bytearray(message)
    elif not isinstance(message, bytearray):
        raise TypeError

    # Padding
    length = len(message) * 8 # len(message) is number of BYTES!!!
    message.append(0x80)
    while (len(message) * 8 + 64) % 512 != 0:
        message.append(0x00)

    message += length.to_bytes(8, 'big') # pad to 8 bytes or 64 bits

    assert (len(message) * 8) % 512 == 0, "Padding did not complete properly!"

    # Parsing
    blocks = [] # contains 512-bit chunks of message
    for i in range(0, len(message), 32): # 64 bytes is 512 bits
        # print(message)
        blocks.append(message[i:i+32])

    # Setting Initial Hash Value
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h5 = 0x9b05688c
    h4 = 0x510e527f
    
    # Hash Computation
    for message_block in blocks:
        # Prepare message schedule
        message_schedule = []
        for t in range(0, 32):
            if t <= 15:
                # adds the t'th 32 bit word of the block,
                # starting from leftmost word
                # 4 bytes at a time
                message_schedule.append(bytes(message_block[t*4:(t*4)+4]))
            else:
                term1 = _sigma1(int.from_bytes(message_schedule[t-2], 'big'))
                term2 = int.from_bytes(message_schedule[t-7], 'big')
                term3 = _sigma0(int.from_bytes(message_schedule[t-15], 'big'))
                term4 = int.from_bytes(message_schedule[t-16], 'big')

                # append a 4-byte byte object
                schedule = ((term1 + term2 + term3 + term4) % 2**32).to_bytes(4, 'big')
                message_schedule.append(schedule)

        assert len(message_schedule) == 32

        # Initialize working variables
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        

        # Iterate for t=0 to 32
        for t in range(32):
            t1 = ((f + _capsigma1(d) + _ch(d,e,f) + K[t] +
                   int.from_bytes(message_schedule[t], 'big')) % 2**32)

            t2 = (_capsigma0(a) + _maj(a, b, c)) % 2**32

            f = e
            e = d
            d = (c + t1) % 2**32
            c = b
            b = a
            a = (t1 + t2) % 2**32

        # Compute intermediate hash value
        h0 = (h0 + a) % 2**32
        h1 = (h1 + b) % 2**32
        h2 = (h2 + c) % 2**32
        h3 = (h3 + d) % 2**32
        h4 = (h4 + e) % 2**32
        h5 = (h5 + f) % 2**32
       

    return ((h0).to_bytes(4, 'big') + (h1).to_bytes(4, 'big') +
            (h2).to_bytes(4, 'big') + (h3).to_bytes(4, 'big') +
            (h4).to_bytes(4, 'big') + (h5).to_bytes(4, 'big')) 

def _sigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 7) ^
           _rotate_right(num, 18) ^
           (num >> 3))
    return num

def _sigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 17) ^
           _rotate_right(num, 19) ^
           (num >> 10))
    return num

def _capsigma0(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 2) ^
           _rotate_right(num, 13) ^
           _rotate_right(num, 22))
    return num

def _capsigma1(num: int):
    """As defined in the specification."""
    num = (_rotate_right(num, 6) ^
           _rotate_right(num, 11) ^
           _rotate_right(num, 25))
    return num

def _ch(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (~x & z)

def _maj(x: int, y: int, z: int):
    """As defined in the specification."""
    return (x & y) ^ (x & z) ^ (y & z)

def _rotate_right(num: int, shift: int, size: int = 32):
    """Rotate an integer right."""
    return (num >> shift) | (num << size - shift)

if __name__ == "__main__":
    a = generate_hash("Pradhikshen").hex()
    print("The hash for sha-192 is "+a)
    print("The length of sha-192 hash is "+str(len(a)))

The hash for sha-192 is f42875fe68ada6b90a51b0c2696751329858f74f19932166
The length of sha-192 hash is 48


In [ ]:
import cv2

img_path1 = '/content/Screenshot (5).png'
img1 = cv2.imread(img_path1, 0) 

(thresh, blackAndWhiteImage) = cv2.threshold(img1, 127, 255, cv2.THRESH_BINARY)
img_reverted1= cv2.bitwise_not(img1)
new_img1 = img_reverted1 #/ 255.0
print(len(new_img1[0]))

result1 = new_img1.flatten()
print(len(result1))
print(result1)

text1 = []
count1 = 0
block1=''
for i in result1:
  count1 +=1
  block1 += str(i)
  if(count1 == 512): 
    text1.append(block1)
    block1 = ''
    count1 = 0
# print(text)
print(len(text1))

# for i in range(0, len(text1)):
  # print('Hash no: ' +str(1)+' is '+ generate_hash(text1[1]).hex())
print(generate_hash(text1[0]).hex())
print(generate_hash(text1[25]).hex())

808
643168
[227 227 227 ... 234 227 227]
1256
bb0bb674a4e02be2c912c9e4156ed42891109060a9aebc55
b9b3e98515bf66ba47e69f58d0f4855ccf2927cdab8937c5


In [ ]:
img_path2 = '/content/Screenshott.jpg'
img2 = cv2.imread(img_path2, 0) 

(thresh, blackAndWhiteImage) = cv2.threshold(img2, 127, 255, cv2.THRESH_BINARY)
img_reverted2= cv2.bitwise_not(img2)
new_img2 = img_reverted2 #/ 255.0
print(len(new_img2[0]))

result2 = new_img2.flatten()
print(len(result2))
print(result2)

text2 = []
count2 = 0
block2=''
for i in result2:
  count2 +=1
  block2 += str(i)
  if(count2 == 512): 
    text2.append(block2)
    block2 = ''
    count2 = 0
# print(text)
print(len(text2))

# for i in range(0, len(text2)):
#   print('Hash no: ' +str(i+1)+' is '+ generate_hash(text2[i]).hex())
print(generate_hash(text2[0]).hex())
print(generate_hash(text2[25]).hex())


528
308352
[221 221 221 ... 232 231 231]
602
1aa8445f7b89bdc92dd0146d2a1042c9fde5fcc4ee510a55
28899e5bc7cd3f0ceea3d8c478b527c98f94b3a916fcee90


In [ ]:
final1 =''
for i in result1:
  final1 += str(i)
print( generate_hash(str(i)).hex())

final2 =''
for i in result2:
  final2 += str(i)
print( generate_hash(str(i)).hex())

32871824ad4240acc3ad83dbdfd05b116220fafd006a129a
b3f353deb07edab3648502f24b28a44dc10f5e4091dc49e4
